In [1]:
from omero.gateway import BlitzGateway
from data_preperation import *

In [2]:
import logging

import numpy as np
from omero.gateway import BlitzGateway, DatasetWrapper, ImageWrapper, ProjectWrapper, FileAnnotationWrapper
import microscopemetrics_schema.datamodel as mm_schema
from linkml_runtime.loaders import yaml_loader

from omero_metrics.tools import omero_tools




In [3]:
conn = BlitzGateway("Asterix", "abc123",group='IGH-Z3', host="localhost", port=6064, secure=True)
conn.connect()

True

In [ ]:
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

# Check if you are an Administrator
print("   Is Admin:", conn.isAdmin())
if not conn.isFullAdmin():
    # If 'Restricted Administrator' show privileges
    print(conn.getCurrentAdminPrivileges())

print("Member of:")
for g in conn.getGroupsMemberOf():
    print("   ID:", g.getName(), " Name:", g.getId())
group = conn.getGroupFromContext()
print("Current group: ", group.getName())

# List the group owners and other members
owners, members = group.groupSummary()
print("   Group owners:")
for o in owners:
    print("     ID: %s %s Name: %s" % (
        o.getId(), o.getOmeName(), o.getFullName()))
print("   Group members:")
for m in members:
    print("     ID: %s %s Name: %s" % (
        m.getId(), m.getOmeName(), m.getFullName()))

print("Owner of:")
for g in conn.listOwnedGroups():
    print("   ID: ", g.getName(), " Name:", g.getId())

# Added in OMERO 5.0
print("Admins:")
for exp in conn.getAdministrators():
    print("   ID: %s %s Name: %s" % (
        exp.getId(), exp.getOmeName(), exp.getFullName()))

# The 'context' of our current session
ctx = conn.getEventContext()
# print(ctx)     # for more info

In [ ]:
for group in conn.listOwnedGroups():
    owners, members = group.groupSummary()
    print("   Group owners:")
    for o in owners:
        print("     ID: %s %s Name: %s" % (
            o.getId(), o.getOmeName(), o.getFullName()))
    print("   Group members:")
    for m in members:
        print("     ID: %s %s Name: %s" % (
            m.getId(), m.getOmeName(), m.getFullName()))

In [ ]:
roi_ids = [r.getId().getValue() for r in result.rois]

In [ ]:
image = conn.getObject("Image", 1)

In [ ]:
image

In [ ]:
dataset = image.getParent()

In [ ]:
analysis_type = get_analysis_type(image.getParent().getParent())

In [ ]:
analysis_type

In [ ]:
roi_service = conn.getRoiService()
result = roi_service.findByImage(int(3), None, conn.SERVICE_OPTS)
shapes_rectangle, shapes_line = get_rois_omero(result)
df_lines_omero = get_info_roi_lines(shapes_line)

In [ ]:
for i in conn.listOwnedGroups():
    print(f'------------------------------{i.getName()}----------------------------------')
    conn.SERVICE_OPTS.setOmeroGroup(i.getId())
    for project in conn.getObjects("Project"):
        print(project.getName(), project.getId())

In [ ]:

shapes_line = {}
shapes_rectangle = {}
for roi in result.rois:
    for s in roi.copyShapes():
        shape = {}
        shape["id"] = s.getId().getValue()
        if s.getTextValue():
            shape["textValue"] = s.getTextValue().getValue()
        if s.__class__.__name__ == "RectangleI":
            shape["type"] = "Rectangle"
            shape["x"] = s.getX().getValue()
            shape["y"] = s.getY().getValue()
            shape["w"] = s.getWidth().getValue()
            shape["h"] = s.getHeight().getValue()
            shapes_rectangle[s.getId().getValue()] = shape
        elif s.__class__.__name__ == "LineI":
            shape["type"] = "Line"
            shape["x1"] = s.getX1().getValue()
            shape["x2"] = s.getX2().getValue()
            shape["y1"] = s.getY1().getValue()
            shape["y2"] = s.getY2().getValue()
            shapes_line[s.getId().getValue()] = shape
        elif s.__class__.__name__ == "PolygonI":
            continue


In [ ]:
g = conn.listProjects()

In [ ]:
for project in conn.getObjects("Project", opts={'group': 4}):
    print(project.getName(), project.getId())

In [ ]:
def get_data_ids(conn):
    for i in conn.listOwnedGroups():
        conn.SERVICE_OPTS.setOmeroGroup(i.getId())
        for project in conn.getObjects("Project"):
            print(project.getName(), project.getId())

In [ ]:
conn.SERVICE_OPTS.setOmeroGroup(-1)
for image in conn.getObjects("Image"):
    print(image.getName(), image.getId())

In [ ]:
dataset_4 = conn.getObject("Dataset", 52)
dataset_6 = conn.getObject("Dataset", 6)


In [ ]:
dataset_4.getAnnotation()

In [ ]:
dataset_4.getAnnotationCounts()

In [ ]:
an= conn.getObject("MapAnnotation", 6)


In [ ]:
ann = dataset_4.getAnnotation()
print("Found Annotation with namespace: ", ann.getNs())

In [ ]:
ann = dataset_4.getAnnotation()
print("Found Annotation with namespace: ", ann.getNs())

In [ ]:
def get_dataset_ids_lists(conn, project_id):
    """
    Get the processed and unprocessed dataset ids for a project
    """
    processed_datasets = []
    unprocessed_datasets = []
    project = conn.getObject("Project", project_id)
    for dataset in project.listChildren():
        try:
            dataset.getAnnotation().getNs()
            processed_datasets.append(dataset.getId())
        except AttributeError:
            unprocessed_datasets.append(dataset.getId())
    return processed_datasets, unprocessed_datasets

In [ ]:
l, n = get_dataset_ids_lists(conn, 3)

In [ ]:
cxt = {'processed_datasets': l, 'unprocessed_datasets': n}

In [ ]:
cxt["processed_datasets"]

In [ ]:
def get_map_annotation(dataset_wrapper, conn):
    df = None
    for i in dataset_wrapper.listAnnotations():
        if i.__class__.__name__ == "MapAnnotationWrapper":
            keyPairs = conn.getObject("MapAnnotation", i.getId())
            table = dict(keyPairs.getValue())
            df = pd.DataFrame(table.items(), columns=["Key", "Value"])
    return df


In [ ]:
def get_dataset_mapAnnotation(conn, dataset_id):
    """
    Get the mapAnnotation for a dataset
    """
    dataset = conn.getObject("Dataset", dataset_id)
    try:
        for i in dataset.listAnnotations():
            if "FieldIlluminationKeyValues" in i.getNs():
                table = dict(i.getValue())
                df = pd.DataFrame(table.items(), columns=["Key", "Value"])
                break
        return df
    except:
        return None

In [ ]:
get_dataset_mapAnnotation(conn, 52)

In [ ]:
for i in dataset_4.listAnnotations():
    print(i.getNs.getValue())

In [ ]:
import pandas as pd

In [ ]:
a = np.arange(12).reshape(3, 4)
np.fliplr(a).diagonal()  # Horizontal flip


In [ ]:
import numpy as np

In [ ]:
np.fliprl(a).diagonal()  # Vertical flip


In [ ]:
b = np.flip(a, axis=1)
print(b[:,-1])
print(b[:,0])
print(np.fliplr(b).diagonal())
print(np.fliplr(a).diagonal())

In [ ]:
np.fliplr(b).diagonal()

In [ ]:
b[:,-1]

In [ ]:
b

In [ ]:
from .load import *

In [ ]:
df_X

In [ ]:
def get_table_originalFile_id(conn,file_id):
    ctx = conn.createServiceOptsDict()
    ctx.setOmeroGroup("-1")
    r = conn.getSharedResources()
    t = r.openTable(omero.model.OriginalFileI(file_id), ctx)   
    data_buffer = collections.defaultdict(list)
    heads = t.getHeaders()
    target_cols = range(len(heads))
    index_buffer = []
    num_rows = t.getNumberOfRows()
    for start in range(0, num_rows):
        data = t.read(target_cols, start, start)
        for col in data.columns:
            data_buffer[col.name] += col.values
        index_buffer += data.rowNumbers
    df = pd.DataFrame.from_dict(data_buffer)
    df.index = index_buffer[0: len(df)]
    return df

In [ ]:
df_X = get_table_originalFile_id(conn,121)
df_Y = get_table_originalFile_id(conn,120)
df_Z = get_table_originalFile_id(conn,119)
df_P = get_table_originalFile_id(conn,118)

In [ ]:
df_P

In [ ]:
df_meta = pd.DataFrame(data=[[int(col.split("_")[-2]), int(col.split("_")[-4]), col.split("_")[-1], col] for col in df_X.columns], columns=['bead_nb', 'channel_nb', 'type', 'name'])

In [ ]:
for col in df_X.columns:
    s = df_X[col]

In [ ]:
df = df_X.copy()

In [ ]:
df_meta

In [ ]:
b ='PSF-beads_60x_2021-03-01_ch_00_bead_00_raw'.split('_')

In [ ]:
df_X

In [ ]:

def get_rois_omero(result):
    shapes_line = {}
    shapes_rectangle = {}
    shapes_point = {}
    for roi in result.rois:
        for s in roi.copyShapes():
            shape = {}
            shape["id"] = s.getId().getValue()
           # shape["theT"] = s.getTheT().getValue()
           # shape["theZ"] = s.getTheZ().getValue()
            if s.getTextValue():
                shape["textValue"] = s.getTextValue().getValue()
            if s.__class__.__name__ == "RectangleI":
                shape["type"] = "Rectangle"
                shape["x"] = s.getX().getValue()
                shape["y"] = s.getY().getValue()
                shape["w"] = s.getWidth().getValue()
                shape["h"] = s.getHeight().getValue()
                shapes_rectangle[s.getId().getValue()] = shape
            elif s.__class__.__name__ == "LineI":
                shape["type"] = "Line"
                shape["x1"] = s.getX1().getValue()
                shape["x2"] = s.getX2().getValue()
                shape["y1"] = s.getY1().getValue()
                shape["y2"] = s.getY2().getValue()
                
                shapes_line[s.getId().getValue()] = shape
            elif s.__class__.__name__ == "PointI":
                shape['type'] = 'Point'
                shape['x'] = s.getX().getValue()
                shape['y'] = s.getY().getValue()
                shape['z'] = s.getTheZ().getValue()
                shape['channel'] = s.getTheC().getValue()
                shapes_point[s.getId().getValue()] = shape
            elif s.__class__.__name__ == "PolygonI":
                continue
    return shapes_rectangle, shapes_line, shapes_point



In [ ]:
get_rois_omero()

In [ ]:
roi_service = conn.getRoiService()
result = roi_service.findByImage(int(20), None, conn.SERVICE_OPTS)
shapes_rectangle, shapes_line, shapes_point = get_rois_omero(result)
df_points_omero = get_info_roi_points(shapes_point)

In [ ]:
df_points_omero.groupby('Z').count().idxmax().ROI

In [ ]:
df_g

In [ ]:
df_points_omero

In [ ]:
def get_info_roi_points(shape_dict):
    data = [
        [key, int(value["x"]), int(value["y"]),int(value["z"]), int(value["channel"]) ]
        for key, value in shape_dict.items()
    ]
    df = pd.DataFrame(data, columns=["ROI", "X", "Y", "Z", 'C'])
    return df

In [50]:
dataset3 = conn.getObject("Dataset", 3).getParent().getId()
dataset3

AttributeError: 'NoneType' object has no attribute 'getParent'

In [5]:
coll = load_project(conn, 1)

In [4]:
from load import load_project

In [ ]:
for i in coll['datasets']:
    #i['processed']
    #i['data_reference']["omero_object_id"]
    #i['data_reference']["omero_object_type"]
    #i['input]["field_illumination_image"]['data_reference']["omero_object_id"]
    print(i['output']['intensity_profiles'][0]['data_reference']["omero_object_id"])
    print(i['output']['key_values'])

In [ ]:
data_reference:
  data_uri: https://localhost:6064/webclient/?show=image-69
  omero_host: localhost
  omero_port: 6064
  omero_object_type: IMAGE
  omero_object_id: 69

In [ ]:
print(f"conn.getGroupFromContext(): {conn.getGroupFromContext().getId()}")
print(f"conn.SERVICE_OPTS.getOmeroGroup(): {conn.SERVICE_OPTS.getOmeroGroup()}")
 

In [165]:
import microscopemetrics_schema.datamodel as mm_schema



In [ ]:
for i in conn.listGroups():
    print(i.getId())

In [9]:
def get_project_data(collections: mm_schema.MetricsDatasetCollection) -> pd.DataFrame:
    data = []
    for dataset in collections.datasets:
        data.append([dataset.__class__.__name__, dataset.data_reference.omero_object_type, dataset.data_reference.omero_object_id, dataset.processed, dataset.acquisition_datetime])
    df = pd.DataFrame(data, columns=["Analysis_type", "Omero_object_type", "Omero_object_id", "Processed", "Acquisition_datetime"])
    return df

In [10]:
df = get_project_data(coll)
df['Omero_object_type'][0]

(text='DATASET', description='A dataset')

In [ ]:
for i in coll.datasets:
    print(type(i.data_reference.omero_object_type))

In [12]:
def get_dataset_data(collections: mm_schema.MetricsDatasetCollection, dataset_id) -> mm_schema.MetricsDataset:
    try:
        dataset = [i for i in collections.datasets if i.data_reference.omero_object_id == dataset_id][0]
        return dataset
    except:
        return None


In [13]:
b = get_dataset_data(coll, 59)

In [24]:
b.processed

True

In [ ]:
def get_dataset_data_dash(dataset: mm_schema.MetricsDataset) -> pd.DataFrame:
    if dataset.processed:
        

In [15]:
def get_images_intensity_profilers(dataset: mm_schema.MetricsDataset) -> pd.DataFrame:
    data=[]
    for i,j in zip(dataset.input['field_illumination_image'],dataset.output["intensity_profiles"]):
        data.append([i['data_reference']['omero_object_id'], j['data_reference']['omero_object_id'], i['shape_c']])
    df = pd.DataFrame(data, columns=["Field_illumination_image", "Intensity_profiles", "Channel"])
    return df

In [ ]:
for i in b.output["intensity_profiles"]:
    print()

In [17]:
b1 = get_images_intensity_profilers(b)

In [172]:
image_77 = conn.getObject("Image", 77)
image_76 = conn.getObject("Image", 76)
image_75 = conn.getObject("Image", 75)

In [34]:
from load import load_image

In [174]:
image_77_na = load_image(image_77)
image_76_na = load_image(image_76)
image_75_na = load_image(image_75)


In [ ]:
image_77_na.shape

In [ ]:
np.insert(image_77_na, image_76_na, axis=4)

In [ ]:
import numpy as np

# Assuming array1 has shape (1, 1, 512, 812, 2) and array2 has shape (1, 1, 512, 812, 1)
array2_padded = np.pad(image_75_na, ((0, 0), (0, 0), (0, 0), (0, 0), (0, 1)), 'constant')


# Now array2_padded has shape (1, 1, 512, 812, 2), and you can add the two arrays together
result = image_77_na + array2_padded

In [ ]:
result.shape

In [35]:
def test(conn, df):
    image_0 = conn.getObject('Image',df['Field_illumination_image'][0])
    image_array_0 = load_image(image_0)
    result = image_array_0
    for i in range(1, len(df)):
        image = conn.getObject('Image', df['Field_illumination_image'][i])
        image_array = load_image(image)
        result = np.concatenate((result,image_array), axis=-1)
    return result
    

In [36]:
images = test(conn, b1 )

In [37]:
images.shape

(1, 1, 512, 812, 4)

In [ ]:
result = np.concatenate((image_77_na, image_76_na), axis=-1)


In [177]:
images.shape

(1, 1, 512, 812, 4)

In [ ]:
def get_intensity_profilers(conn, df):
    
        
    

In [194]:
for i in b1.Intensity_profiles:
    file_id = conn.getObject('FileAnnotation', i).getFile().getId()
    print(get_table_originalFile_id(conn,file_id))

     ch00_leftTop_to_rightBottom  ch00_leftBottom_to_rightTop  \
0                        31893.0                      35632.0   
1                        31956.0                      35567.0   
2                        32085.0                      35748.0   
3                        32193.0                      35937.0   
4                        32329.0                      35923.0   
..                           ...                          ...   
250                          0.0                          0.0   
251                          0.0                          0.0   
252                          0.0                          0.0   
253                          0.0                          0.0   
254                          0.0                          0.0   

     ch00_center_horizontal  ch00_center_vertical  
0                   36745.0               33667.0  
1                   36915.0               33747.0  
2                   36961.0               33831.0  
3          

In [18]:
conn.getObject('FileAnnotation', 131).getFile().getId()

244

In [19]:
b1

,Field_illumination_image,Intensity_profiles,Channel
0,75,131,1
1,76,132,1
2,77,133,2


In [21]:
df

,Analysis_type,Omero_object_type,Omero_object_id,Processed,Acquisition_datetime
0,FieldIlluminationDataset,[_code],59,True,2021-03-04T00:00:00
1,FieldIlluminationDataset,[_code],60,True,2021-05-04T00:00:00
2,FieldIlluminationDataset,[_code],61,True,2021-07-04T00:00:00
3,FieldIlluminationDataset,[_code],62,True,2021-09-04T00:00:00


In [55]:
project_id = conn.getObject("Dataset", 1).getParent().getId()

In [56]:
project_id

1

In [23]:
from load import get_dataset_by_id, get_key_values

ImportError: cannot import name 'get_key_values' from 'load' (/home/wapaa/projects/OMERO-metrics/omero_metrics/tools/load.py)

In [6]:
coll = load_project(conn, 1)

In [11]:
dataset = get_dataset_by_id(coll, 1)

In [12]:
dataset.processed

True

In [77]:
project_id = conn.getObject("Dataset", 3).getParent().getId()
collections_mm_p = load_project(conn, project_id)
#dataset = load.get_dataset_by_id(collections_mm_p, dataset_id)
dataset = [i for i in collections_mm_p.datasets if i.data_reference.omero_object_id == 3][0]
dataset

{
    map = 
    {
        key = ids
        value = object #1 (::omero::RList)
        {
            _val = 
            {
                [0] = object #2 (::omero::RLong)
                {
                    _val = 3
                }
            }
        }
    }
    theFilter = <nil>
    theOptions = <nil>
}, <ServiceOptsDict: {'omero.client.uuid': '2bd3ca93-3101-4ed1-b4c9-d39836a36190', 'omero.event': 'Internal', 'omero.session.uuid': 'cec2368c-1bba-4956-a732-373d194f90e2'}>), {})
Traceback (most recent call last):
  File "/home/wapaa/projects/OMERO-metrics/.venv/lib/python3.11/site-packages/omero/gateway/__init__.py", line 4810, in __call__
    return self.f(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/wapaa/projects/OMERO-metrics/.venv/lib/python3.11/site-packages/omero_api_IQuery_ice.py", line 651, in findByQuery
    return _M_omero.api.IQuery._op_findByQuery.invoke(self, ((query, params), _ctx))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

ConnectionLostException: Ice.ConnectionLostException:
recv() returned zero

In [76]:
for i in collections_mm_p.datasets:
    print(i.data_reference.omero_object_id)

3
4
5
6


SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3720334485.py, line 1)

In [16]:
if dataset.processed:
    print('processed')
else:
    print('unprocessed')

processed


In [17]:
dataset.output.key_values

FieldIlluminationKeyValues(name=None, description=None, data_reference=DataReference(data_uri='https://localhost:6064/webclient/?show=key_values-3', omero_host='localhost', omero_port=6064, omero_object_type=(text='KEY_VALUES', description='A set of key-value pairs'), omero_object_id=3), linked_references=[], channel_name=['DAPI', 'FITC', 'TRITC', 'CY5'], center_region_intensity_fraction=[0.05, 0.05, 0.05, 0.05], center_region_area_fraction=[0.1268310546875, 0.159332275390625, 0.14093017578125, 0.1522064208984375], center_of_mass_y=[236.0090531761309, 297.0, 292.0, 204.0104260651629], center_of_mass_y_relative=[-0.07808963603073871, 0.16015625, 0.140625, -0.2030842731829574], center_of_mass_x=[236.99636068334937, 227.0, 287.0, 251.00689223057645], center_of_mass_x_relative=[-0.07423296608066654, -0.11328125, 0.12109375, -0.019504327224310725], center_of_mass_distance_relative=[0.10774286291233695, 0.19616999264827686, 0.18557771125343286, 0.2040187265785253], center_geometric_y=[236.00

In [18]:
dataset.output

FieldIlluminationOutput(processing_application=['microscopemetrics'], processing_version=['0.1.0'], processing_datetime='2024-05-06T15:27:17.221153', processing_entity=[], processing_log=None, warnings=[], errors=[], comment=None, key_values=FieldIlluminationKeyValues(name=None, description=None, data_reference=DataReference(data_uri='https://localhost:6064/webclient/?show=key_values-3', omero_host='localhost', omero_port=6064, omero_object_type=(text='KEY_VALUES', description='A set of key-value pairs'), omero_object_id=3), linked_references=[], channel_name=['DAPI', 'FITC', 'TRITC', 'CY5'], center_region_intensity_fraction=[0.05, 0.05, 0.05, 0.05], center_region_area_fraction=[0.1268310546875, 0.159332275390625, 0.14093017578125, 0.1522064208984375], center_of_mass_y=[236.0090531761309, 297.0, 292.0, 204.0104260651629], center_of_mass_y_relative=[-0.07808963603073871, 0.16015625, 0.140625, -0.2030842731829574], center_of_mass_x=[236.99636068334937, 227.0, 287.0, 251.00689223057645], 

In [22]:
import get_key_values from load

SyntaxError: invalid syntax (1037727524.py, line 1)

In [24]:
var = dataset.output

In [26]:
data_dict = var.key_values.__dict__
data_dict = {
    key: value[0] if isinstance(value, list) and value else value
    for key, value in data_dict.items()
}
data_list = list(data_dict.items())
df = pd.DataFrame(data_list, columns=["Key", "Value"])
df.Key = df.Key.str.replace("_", " ").str.title()